Для начала приведем csv файл в корекнтный формат

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


input_file = "/content/drive/MyDrive/final_train.csv"
df = pd.read_csv(input_file)

In [3]:
df.head()
len(df)

446559

In [4]:
texts = df['text'].values
for i in range(len(texts)):
    texts[i] = "clustering: " + texts[i]

emojis_data = df['emoji'].factorize()
emojis = emojis_data[0]
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, emojis, test_size=0.1, random_state=42)

In [5]:
emojis_data[0][:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 0])

In [6]:
map = {}
for i in range(len(emojis_data[0])):
    em = emojis_data[0][i]
    if em not in map:
        map[em] = 1
    else:
        map[em] += 1

print(len(map))

62


In [7]:
def emoji_to_num(emoji):
  for index, elem in enumerate(emojis_data[1]):
    if elem == emoji:
      return index

def num_to_emoji(num):
  return emojis_data[1][num]

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained("ai-forever/ru-en-RoSBERTa")
tokenizer.truncation_side = 'left'
bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_data(texts, tokenizer, max_len=24):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

In [10]:
train_encodings = tokenize_data(train_texts, tokenizer)
val_encodings = tokenize_data(val_texts, tokenizer)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


Создаем пайторч датасет

In [11]:
import torch

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

train_dataset = EmojiDataset(train_encodings, train_labels)
val_dataset = EmojiDataset(val_encodings, val_labels)


In [12]:
def tokenize_data2(texts, tokenizer, max_len=24):
    return tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
test = ["здарова sdgd sdgdfg",
        "привет"]
emb_test = tokenize_data2(test, tokenizer)
print(emb_test)
emb_test2 = bert(**emb_test)
emb_test2.last_hidden_state.shape

{'input_ids': tensor([[    1,  4371, 18907, 94754, 75638, 94754,    75, 85240,    75,     2],
        [    1,   479,  5946,     2,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}


torch.Size([2, 10, 1024])

Создаем модель

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        probs = F.softmax(inputs, dim=-1)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
        probs = torch.sum(probs * targets_one_hot, dim=-1)

        focal_weight = (1 - probs) ** self.gamma

        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        loss = focal_weight * ce_loss

        if self.alpha is not None:
            alpha_weight = self.alpha.gather(0, targets)
            loss = alpha_weight * loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


find alphas

In [14]:

num_classes = 62
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

alpha = torch.zeros(num_classes).to(device)
for i in range(len(alpha)):
  alpha[i] = 8000 / map[i]


alpha2 = torch.ones(num_classes).to(device)

alpha3 = torch.sqrt(alpha)

alpha4 = torch.pow(alpha, 3/4)




In [15]:
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 17.1 MB/s eta 0:00:00


In [16]:
import torch.nn as nn
from transformers import BertModel
from transformers import AutoModel
from peft import LoraConfig, get_peft_model
import torch.nn.functional as F

class EmojiClassifier2(nn.Module):
    def __init__(self, num_labels, alpha):
        super(EmojiClassifier2, self).__init__()
        self.bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

        lora_config = LoraConfig(
            r=32,
            lora_alpha=32,
            target_modules=["query", "value", "key"],
            bias="none"
        )

        self.bert = get_peft_model(self.bert, lora_config)
        self.alpha = alpha

        for param in self.bert.parameters():
            param.requires_grad = False
        for name, param in self.bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True


        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, attention_mask, labels=None): # тут посмотреть
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)

        if labels is not None:
            loss_fct = FocalLoss(gamma=1.0, alpha=alpha3)
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits

In [2]:
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import LambdaLR


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = EmojiClassifier2(num_labels=len(df['emoji'].unique()), alpha=alpha3).to(device)

model_load_path = '/content/drive/MyDrive/model32lora2024-10-04_23:28:31.pth'
model.load_state_dict(torch.load(model_load_path, map_location=device))

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

optimizer = AdamW(model.parameters(), lr=2e-4)


NameError: name 'torch' is not defined

In [18]:
optimizer = AdamW(model.parameters(), lr=4e-5)


In [19]:
def lr_lambda(current_step: int):
    warmup_steps = 50  # Количество шагов для warmup
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)


In [20]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np
from datetime import datetime

from torch.utils.tensorboard import SummaryWriter

def train_model(model, train_loader, val_loader, epochs=10):
    writer = SummaryWriter()

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')
        now = datetime.now()

        formatted_date_time = now.strftime("%Y-%m-%d_%H:%M:%S")
        model_save_path = f'/content/drive/MyDrive/model32lora{formatted_date_time}.pth'
        torch.save(model.state_dict(), model_save_path)


        model.eval()
        correct = 0
        total = 0
        correct_top10 = 0
        class_total = np.zeros(num_classes)
        class_correct = np.zeros(num_classes)
        class_predicted = np.zeros(num_classes)


        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == batch['labels']).sum().item()
                total += len(batch['labels'])

                top10_preds = torch.topk(logits, k=10, dim=-1).indices
                correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1


        top1_acc = correct / total * 100
        top10_acc = correct_top10 / total * 100

        writer.add_scalar('Accuracy/Top-1', top1_acc, epoch)
        writer.add_scalar('Accuracy/Top-10', top10_acc, epoch)

        print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')

    writer.close()




train_model(model, train_loader, val_loader)

Epoch [1/10]: 100%|██████████| 1570/1570 [1:00:00<00:00,  2.29s/it, loss=2.92]


Epoch 1, Loss: 2.954104176600268


Validating: 100%|██████████| 698/698 [03:39<00:00,  3.18it/s]


Validation Accuracy (Top-1): 19.51%
Validation Accuracy (Top-10): 69.27%
Class 0 Recall: 32.87% (661/2011)      Precision: 23.93% (661/2762)
Class 1 Recall: 7.97% (31/389)      Precision: 6.29% (31/493)
Class 2 Recall: 39.89% (446/1118)      Precision: 18.64% (446/2393)
Class 3 Recall: 9.97% (35/351)      Precision: 9.75% (35/359)
Class 4 Recall: 4.96% (76/1531)      Precision: 15.38% (76/494)
Class 5 Recall: 6.51% (22/338)      Precision: 8.66% (22/254)
Class 6 Recall: 44.16% (2254/5104)      Precision: 27.75% (2254/8123)
Class 7 Recall: 15.06% (557/3699)      Precision: 18.29% (557/3046)
Class 8 Recall: 14.45% (98/678)      Precision: 16.58% (98/591)
Class 9 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 215.0, class_correct 0.0
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 7.46% (139/1863)      Precision: 14.21% (139/978)
Class 12 Recall: 2.67% (25/936)      Precision:

Epoch [2/10]: 100%|██████████| 1570/1570 [59:58<00:00,  2.29s/it, loss=3.05]


Epoch 2, Loss: 2.9431396069800018


Validating: 100%|██████████| 698/698 [03:38<00:00,  3.19it/s]


Validation Accuracy (Top-1): 19.44%
Validation Accuracy (Top-10): 69.45%
Class 0 Recall: 34.26% (689/2011)      Precision: 24.12% (689/2857)
Class 1 Recall: 7.97% (31/389)      Precision: 7.43% (31/417)
Class 2 Recall: 43.74% (489/1118)      Precision: 17.33% (489/2822)
Class 3 Recall: 13.11% (46/351)      Precision: 9.16% (46/502)
Class 4 Recall: 3.33% (51/1531)      Precision: 16.19% (51/315)
Class 5 Recall: 6.51% (22/338)      Precision: 6.81% (22/323)
Class 6 Recall: 41.91% (2139/5104)      Precision: 27.82% (2139/7688)
Class 7 Recall: 13.19% (488/3699)      Precision: 18.42% (488/2650)
Class 8 Recall: 15.04% (102/678)      Precision: 16.32% (102/625)
Class 9 Recall: 0.00% (0/215)      Precision: 0.00% (0/1)
Class 10 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 387.0, class_correct 0.0
Class 11 Recall: 8.21% (153/1863)      Precision: 14.00% (153/1093)
Class 12 Recall: 1.92% (18/936)      Precision: 8.18% (18/220)
Class 13 Recall: 21.38% (158/739) 

Epoch [3/10]:   1%|          | 11/1570 [00:26<1:02:20,  2.40s/it, loss=2.95]


KeyboardInterrupt: 

In [24]:
now = datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M:%S")
model_save_path = f'/content/drive/MyDrive/model32lora{formatted_date_time}.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
now = datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d_%H:%M:%S")
model_save_path = f'/content/drive/MyDrive/model8lora{formatted_date_time}.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
model_save_path = '/content/drive/MyDrive/checkpoints/model.pth'
torch.save(model.state_dict(), model_save_path)


In [ ]:
model = YourModelClass()

model_load_path = '/content/drive/My Drive/model.pth'
model.load_state_dict(torch.load(model_load_path, map_location=device))


In [ ]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter

def val_model(model, train_loader, val_loader, epochs=10):

    model.eval()
    correct = 0
    total = 0
    correct_top10 = 0
    class_total = np.zeros(num_classes)
    class_correct = np.zeros(num_classes)
    class_predicted = np.zeros(num_classes)


    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch)[1]
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == batch['labels']).sum().item()
            total += len(batch['labels'])

            top10_preds = torch.topk(logits, k=10, dim=-1).indices
            correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

            for i in range(len(batch['labels'])):
                label = batch['labels'][i].item()
                pred = predictions[i].item()
                class_predicted[pred] += 1
                class_total[label] += 1
                if label == pred:
                    class_correct[label] += 1


    top1_acc = correct / total * 100
    top10_acc = correct_top10 / total * 100


    print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
    print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

    for i in range(num_classes):
        if class_total[i] > 0 and class_predicted[i] > 0:
            recall = 100 * class_correct[i] / class_total[i]
            prec = 100 * class_correct[i] / class_predicted[i]
            print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
        else:
            print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')



val_model(model, train_loader, val_loader)

In [ ]:
bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")





In [ ]:

for param in bert.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [ ]:
bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


In [ ]:
bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


In [ ]:
442368 / 1179648

In [ ]:
bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
